## Prepare input structures

In [1]:
import glob
from dotenv import load_dotenv
from ase.io import read
from mlip_arena.tasks.mof.input import save_to_db

load_dotenv()

files = glob.glob('structures/*.cif')

atoms_list = []

for file in files:
    atoms = read(file)
    atoms_list.append(atoms)

save_to_db(
    atoms_list=atoms_list,
    db_path='mofs.db'
)

No module named 'deepmd'


/pscratch/sd/c/cyrusyc/.conda/mlip-arena/lib/python3.11/site-packages/ase/io/cif.py:408: UserWarning: crystal system 'triclinic' is not interpreted for space group Spacegroup(1, setting=1). This may result in wrong setting!
  warnings.warn(


mofs.db uploaded to atomind/mlip-arena/mof


## Flow definition

In [1]:
from ase.build import molecule
from mlip_arena.models import MLIPEnum
from mlip_arena.tasks.mof.input import get_atoms_from_db
from mlip_arena.tasks.mof.flow import widom_insertion
from tqdm.auto import tqdm

from prefect import flow

@flow
def benchmark_test():
    results = []
    for model in MLIPEnum:
        if model.name != "MACE-MP(M)":
            continue
        for atoms in tqdm(get_atoms_from_db('mofs.db')):
            result = widom_insertion(
                atoms, 
                molecule('CO2'),
                calculator_name=model.name,
            )
            results.append(result)
            break
    
    return [r for r in results]

No module named 'deepmd'


15:57:19.295 | WARNING | MDAnalysis.coordinates.AMBER - netCDF4 is not available. Writing AMBER ncdf files will be slow.

In [ ]:

result = benchmark_test()

print(result)



In [10]:
result[0]['henry_coefficient']

[2.3617881373330576e-06, 2.5985147795206755e-06]